In [ ]:
%pip install "zenml[server]"
%pip install zenml
!zenml integration install sklearn -y
%pip install pyparsing
import IPython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 kB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

NumExpr defaulting to 2 threads.
Initializing the ZenML global configuration version to 0.55.2
⠧ Installing integrations...


In [ ]:
NGROK_TOKEN = "----------"
from zenml.environment import Environment

if Environment.in_google_colab():
  !pip install pyngrok
  !ngrok authtoken {NGROK_TOKEN}

!rm -rf .zen
!zenml init


INFO:numexpr.utils:NumExpr defaulting to 2 threads.


NumExpr defaulting to 2 threads.
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
NumExpr defaulting to 2 threads.
Creating database tables
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
Creating default workspace 'default' ...
Creating default stack in workspace default...
Creating default user 'default' ...
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
Setting the global active workspace to 'default'.
Setting the global active stack to default.
⠸ Initializing ZenML repository at /content.
Setting the repo active workspace to 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
import numpy as np
from typing import Tuple

from zenml import step

@step
def import_data_ratings() -> pd.DataFrame:
  ratings_data=pd.read_csv("/content/ratings.csv")
  #ratings_data.info()
  #movies_data.info()
  uuu=ratings_data['userId'].nunique()
  print(f"Il y a {uuu} userId uniques dans ratings_data.")
  return ratings_data



In [ ]:
@step
def import_data_movies() -> pd.DataFrame:
  movies_data=pd.read_csv("/content/movies.csv")
  return movies_data

In [ ]:
@step
def preprocessing(ratings_data: pd.DataFrame, movies_data: pd.DataFrame)-> pd.DataFrame:
  # Merge ratings and movies data
  merged_data = pd.merge(ratings_data, movies_data, on='movieId')
  # Drop unnecessary columns
  merged_data = merged_data.drop(['timestamp', 'genres'], axis=1)
  # Create user-item matrix
  user_item_matrix = merged_data.pivot_table(index='userId', columns='movieId', values='rating')
  print("The size of user_item_matrix: ",user_item_matrix.size)
  print("The type of user_item_matrix: ",type(user_item_matrix))
  user_item_matrix.columns = [str(col) for col in user_item_matrix.columns]
  return user_item_matrix

In [ ]:
@step
def split_data( user_item_matrix: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
  train_data, test_data = train_test_split(user_item_matrix, test_size=0.2, random_state=42)
  print(train_data.shape)
  print(test_data.shape)
  print(type(train_data))
  return  train_data, test_data

In [ ]:
@step
def cosinus_trainer(train_data:pd.DataFrame) -> np.ndarray:
  #print(train_data.shape)
  item_similarity = cosine_similarity(train_data.fillna(0))
  np.fill_diagonal(item_similarity, 0)
  print(type(item_similarity))
  print(item_similarity.ndim)
  print(item_similarity)
  return item_similarity

In [ ]:
@step
def corrcoef_trainer(train_data:pd.DataFrame) -> np.ndarray:
  #print(train_data.shape)
  #item_similarity = cosine_similarity(train_data.fillna(0))
  item_similarity = 1 - pairwise_distances(train_data.fillna(0), metric='correlation')
  print(item_similarity)
  np.fill_diagonal(item_similarity, 0)
  print(type(item_similarity))
  print(item_similarity.ndim)
  return item_similarity

In [ ]:
@step
def get_recommendations(user_id: int,train_data:pd.DataFrame, item_similarity:np.ndarray,
                        user_item_matrix:pd.DataFrame,movies_data: pd.DataFrame, top_n:int=5) -> pd.DataFrame:
  # Get the user's ratings and item similarity
  user_ratings = train_data.loc[user_id].values.reshape(1, -1)
  #print(type(user_ratings))
  #print(user_ratings.ndim)
  masque = ~np.isnan(user_ratings)
  user_ratings = user_ratings[masque]
  #print(train_data.shape[0])
  #print(user_ratings.shape)
  # Déterminer le nombre de colonnes pour compléter la matrice jusqu'à 6624 colonnes
  colonnes_manquantes = train_data.shape[0] - user_ratings.shape[0]
  # Complétez la matrice avec des zéros pour atteindre la dimension 1x6624
  matrice_completee = np.pad(user_ratings, (0, colonnes_manquantes), 'constant', constant_values=0)
  user_ratings = matrice_completee
  similarity_scores = item_similarity.dot(user_ratings.T).reshape(-1)
  #print(similarity_scores.shape)

  # Sort the movies based on similarity scores
  sorted_indices = np.argsort(similarity_scores)[::-1]
  top_indices = sorted_indices[:top_n]
  #print(top_indices)

  # Get the recommended movie IDs
  recommended_movie_ids = ((user_item_matrix.columns[top_indices]).astype(int)).tolist()
  #aaaa = ((user_item_matrix.columns[top_indices]).astype(int)).tolist()
  #print(aaaa)
  #bbbb=aaaa.astype(int)
  #bbbb=bbbb.tolist()
  #print(bbbb)
  # Get the movie titles
  recommended_movies = movies_data[movies_data['movieId'].isin(recommended_movie_ids)]

  print(recommended_movies)
  return recommended_movies


In [ ]:
from zenml import pipeline

@pipeline
def reco_pipeline():
  """Links all the steps together in a pipeline"""
  ratings_data = import_data_ratings()
  movies_data = import_data_movies()
  merge_data = preprocessing(ratings_data, movies_data)
  #merged_data = merge_data(import_data_ratings(), import_data_movies())
  train_data, test_data = split_data(merge_data)
  trainer= cosinus_trainer(train_data=train_data)
  evaluation=get_recommendations(input("Veuillez entrer une valeur : "), train_data=train_data, item_similarity=trainer, user_item_matrix=merge_data, movies_data=movies_data)

In [ ]:
from zenml import pipeline

@pipeline
def reco_corrcoef_pipeline():
  """Links all the steps together in a pipeline"""
  ratings_data = import_data_ratings()
  movies_data = import_data_movies()
  merge_data = preprocessing(ratings_data, movies_data)
  #merged_data = merge_data(import_data_ratings(), import_data_movies())
  train_data, test_data = split_data(merge_data)
  trainer= corrcoef_trainer(train_data=train_data)
  evaluation=get_recommendations(input("Veuillez entrer une valeur : "), train_data=train_data, item_similarity=trainer, user_item_matrix=merge_data, movies_data=movies_data)

In [ ]:
reco_cosine_pipeline = reco_pipeline()


Veuillez entrer une valeur : 40
Initiating a new run for the pipeline: reco_pipeline.
Registered new version: (version 1).
Executing a new run.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
Step import_data_movies has started.
Step import_data_movies has finished in 0.921s.
Step import_data_ratings has started.
Il y a 6881 userId uniques dans ratings_data.
Step import_data_ratings has finished in 1.553s.
Step preprocessing has started.
The size of user_item_matrix:  151953123
The type of user_item_matrix:  <class 'pandas.core.frame.DataFrame'>
Metadata value for key 'dtype' is too large to be stored in the database. Skipping.
Metadata value for key 'mean' is too large to be stored in the database. Skipping.
Metadata value for key 'std' is too large to be stored in the database. Skipping.
Metadata value for key 'min' is too large to be stored in the database. Skipping.
Metadata value for key 'max' is too large to be stored in the database. Sk

In [ ]:
!zenml up

In [ ]:
reco_corrcoef_pipeline = reco_corrcoef_pipeline()

Veuillez entrer une valeur : 5
Initiating a new run for the pipeline: reco_corrcoef_pipeline.
Registered new version: (version 1).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
Using cached version of import_data_movies.
Linking artifact output to model None version None implicitly.
Step import_data_movies has started.
Using cached version of import_data_ratings.
Linking artifact output to model None version None implicitly.
Step import_data_ratings has started.
Using cached version of preprocessing.
Linking artifact output to model None version None implicitly.
Step preprocessing has started.
Using cached version of split_data.
Linking artifact output_1 to model None version None implicitly.
Linking artifact output_0 to model None version None implicitly.
Step split_data has started.
Step corrcoef_trainer has started.
[[ 1.          0.0339268   0.09713478 ...  0.29806043  0.09254272
   0.09840934]
 [ 0.0339268   1.     

In [ ]:
!zenml down

NumExpr defaulting to 2 threads.
Tearing down the 'local' local ZenML server.
Removing the local ZenML server.
⠋ Stopping service 'LocalZenServer[675a916c-efd2-4711-a646-5b27bb8bfee2] (type: zen_server, flavor: 
local)'.

The local ZenML dashboard has been shut down.


In [ ]:
from zenml.environment import Environment

def start_zenml_dashboard(port=8237):
  if Environment.in_google_colab():
    from pyngrok import ngrok

    public_url = ngrok.connect(port)
    #print(f"\xlb[31mIn Colab, use this url instead: {public_url}!\xlb]0m")
    print(f"\\x1b[31mIn Colab, use this url instead: {public_url}!\\x1b[0m")
    !zenml up --blocking --port {port}

  else:
    !zenml up --port {port}
start_zenml_dashboard()

INFO:pyngrok.ngrok:Opening tunnel named: http-8237-ff4e67eb-3db5-4c07-a6ab-58a11cc00622


Opening tunnel named: http-8237-ff4e67eb-3db5-4c07-a6ab-58a11cc00622


INFO:pyngrok.process.ngrok:t=2024-02-14T11:43:31+0000 lvl=info msg="no configuration paths supplied"


t=2024-02-14T11:43:31+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2024-02-14T11:43:31+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

t=2024-02-14T11:46:03+0000 lvl=info msg="join connections" obj=join id=9dd88c136ecc l=127.0.0.1:8237 r=37.67.75.184:13511
Got token from cookie
INFO:     37.67.75.184:0 - "GET /api/v1/steps/b4f4053f-0f01-42f2-b27b-8e3422eb736d HTTP/1.1" 200 OK
Got token from cookie
INFO:     37.67.75.184:0 - "GET /api/v1/artifact_versions/65619ced-41b3-46f3-a653-d76f510cf06a HTTP/1.1" 200 OK
Got token from cookie
INFO:     37.67.75.184:0 - "GET /api/v1/artifact_versions/65619ced-41b3-46f3-a653-d76f510cf06a HTTP/1.1" 200 OK
Got token from cookie
INFO:     37.67.75.184:0 - "GET /api/v1/artifact_versions/65619ced-41b3-46f3-a653-d76f510cf06a/visualize HTTP/1.1" 200 OK


INFO:pyngrok.process.ngrok:t=2024-02-14T11:47:54+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


t=2024-02-14T11:47:54+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


INFO:pyngrok.process.ngrok:t=2024-02-14T11:47:54+0000 lvl=info msg="session closing" obj=tunnels.session err=nil


t=2024-02-14T11:47:54+0000 lvl=info msg="session closing" obj=tunnels.session err=nil
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [3020]


In [ ]:
! pip show pydantic
! pip show zenml
! pip show scikit-learn


Name: pydantic
Version: 1.10.14
Summary: Data validation and settings management using python type hints
Home-page: https://github.com/pydantic/pydantic
Author: Samuel Colvin
Author-email: s@muelcolvin.com
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: typing-extensions
Required-by: confection, fastapi, fastapi-utils, inflect, lida, llmx, spacy, sqlmodel, thinc, weasel, zenml
Name: zenml
Version: 0.55.2
Summary: ZenML: Write production-ready ML code.
Home-page: https://zenml.io
Author: ZenML GmbH
Author-email: info@zenml.io
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: alembic, azure-mgmt-resource, bcrypt, click, click-params, cloudpickle, distro, docker, gitpython, httplib2, pandas, passlib, psutil, pydantic, pymysql, pyparsing, python-dateutil, pyyaml, rich, sqlalchemy_utils, sqlmodel
Required-by: 
Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit

In [ ]:
! python --version

Python 3.10.12
